作成：瀧川英輝

In [7]:
;cat lin_interp.jl

In [8]:
include("lin_interp.jl")

my_lin_interp (generic function with 1 method)

簡単な例を試す。

In [9]:
grid = [1, 2]
vals = [2, 0]
f = my_lin_interp(grid, vals)

f(1.25)

1.5

gridの範囲外が与えられた場合は"Error!"と返すようにした。

In [10]:
f(0)

"Error!"

ベクトルの場合は以下のようになる

In [ ]:
f([])